In [3]:
import os
import sys
import json 
import torch 
from torch_geometric.data import Data, DataLoader
import heapq
from dataclasses import dataclass, field
from typing import Any
import random 

common_path = os.path.abspath(os.path.join("__file__", "../../common"))
src_path = os.path.abspath(os.path.join("__file__", "../../"))

sys.argv = ["foo"]
print(common_path)
print(src_path)
sys.path.append(common_path)
sys.path.append(src_path)

from cmd_args import cmd_args, logging
from env import Env
from embedding import GNN, SceneDataset, create_dataset
from utils import get_config
from copy import deepcopy

data_dir = os.path.abspath(os.path.join(os.path.abspath("__file__"), "../../../data"))
print(data_dir)


/Users/jianihuang/Desktop/research/synthesis_specs/src/common
/Users/jianihuang/Desktop/research/synthesis_specs/src
log path :/Users/jianihuang/Desktop/research/synthesis_specs/data/log/5e-05-penyes-preno_intermediate-norno-None.log
Namespace(batch_size=5, beam_size=30, dataset_name='img_test_500.pt', decay=0.95, embedding_dim=64, episode_iter=50000, episode_length=10, eps=0.2, eps_decay=0.99, graph_file_name='img_test_500.pkl', hard_constraint=True, hidden_dim=64, json_path='/Users/jianihuang/Desktop/research/synthesis_specs/data/temp/query-5e-05-penyes-preno_intermediate-norno-None.json', log_name=None, log_path='/Users/jianihuang/Desktop/research/synthesis_specs/data/log/5e-05-penyes-preno_intermediate-norno-None.log', lr=5e-05, max_obj_num=20, max_sub_prob=20, max_var_num=3, model_dir='/Users/jianihuang/Desktop/research/synthesis_specs/data/model', model_path='/Users/jianihuang/Desktop/research/synthesis_specs/data/model/refrl-5e-05-penyes-no_intermediate-norno-None.pkl', normaliz

In [15]:
hard_idxs = ['22', '25', '35', '36', '191', '192', '315', '318', '391', '398', '445', '447', '491', '515', '516', '523', '524', '755', '759', '838', '842', '980', '1048', '1049', '1115', '1161', '1162', '1173', '1174', '1349', '1352', '1399', '1405', '1406', '1464', '1465', '1563', '1565', '1574', '1576', '1607', '1611', '1625', '1626', '1628', '1689', '1690', '1691', '1727', '1775', '1776', '1820', '1821', '1870', '1876', '1900', '1903', '1963', '1964', '1998', '2005', '2093', '2096', '2129', '2131', '2175', '2176', '2210']
hard_idxs = [ int(idx) for idx in hard_idxs]
scenes = []

data_dir = os.path.abspath(os.path.join(os.path.abspath("__file__"), "../../../data"))
model_dir = os.path.abspath(os.path.join(data_dir, "oak_model/model"))
og_dir = os.path.abspath(os.path.join(data_dir, "log"))

scene_file_name = "img_test_500.json"
graph_file_name = "img_test_500.pkl"
datafile_name = "img_test_500.pt"
search_file_name = "beam_search_500.json"

raw_path = os.path.abspath(os.path.join(data_dir, "./processed_dataset/raw"))
scenes_path = os.path.abspath(os.path.join(raw_path, scene_file_name))
graphs_path = os.path.join(raw_path, graph_file_name)
success_path = os.path.join(data_dir, f"./eval_result/{search_file_name}")

graphs, scene_dataset = create_dataset(data_dir, scenes_path, graphs_path)

# update the cmd_args corresponding to the info we have 
cmd_args.graph_file_name = graph_file_name

lr4_10_model_path = os.path.join(model_dir, "refrl-0.0001-penyes-no_intermediate-norno-img_test_30.pkl")
refrl = torch.load(lr4_10_model_path)
refrl.policy.eval()

data_loader = DataLoader(scene_dataset)
for ct, data_point in enumerate(data_loader):
    if ct in hard_idxs:
        graph = graphs[ct]
        graph.scene["target_id"] = graph.target_id
        scenes.append(graph.scene)
            

['large', 'small', 'blue', 'red', 'yellow', 'green', 'gray', 'brown', 'purple', 'cyan', 'cube', 'cylinder', 'sphere', 'rubber', 'metal', 'right', 'behind', 'target', 'obj_0', 'obj_1', 'obj_2', 'obj_3', 'obj_4', 'obj_5', 'obj_6', 'obj_7', 'obj_8', 'obj_9', 'obj_10', 'obj_11', 'obj_12', 'obj_13', 'obj_14', 'obj_15', 'obj_16', 'obj_17', 'obj_18', 'obj_19', 'var_0', 'var_1', 'var_2', 'size', 'color', 'shape', 'material', 'bonding', 'edge_attr', 'first', 'second', 'target', 'center_relation']


In [16]:
hard_file_name = "hard_scenes.json"
hard_path = os.path.join(raw_path, hard_file_name)
with open (hard_path, 'w') as hard_file:
    json.dump(scenes, hard_file, indent=True)

In [14]:
print(graph.target_id)

0
